In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
%store -r X_features_full
%store -r X_features_poly_full
%store -r y_labels_full
X_features = X_features_full
X_features_poly = X_features_poly_full
y_labels = y_labels_full

from sklearn.metrics import mean_absolute_percentage_error


In [2]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR



In [3]:
svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
forest_reg = RandomForestRegressor(n_estimators=150,max_samples=5000)

poly_svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
poly_forest_reg = RandomForestRegressor(n_estimators=100)

voting_reg = VotingRegressor(
    estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)])

poly_voting_reg = VotingRegressor(
    estimators=[('pfr', poly_svm_rbf_reg), ('psvm', poly_forest_reg)])

In [4]:
voting_reg.fit(X_features[0],y_labels[0])
poly_voting_reg.fit(X_features_poly[0],y_labels[0])

VotingRegressor(estimators=[('pfr', SVR(C=1, epsilon=0.001)),
                            ('psvm', RandomForestRegressor())])

In [5]:
from sklearn.metrics import mean_absolute_percentage_error

visc_predictions = np.exp(np.power(poly_voting_reg.predict(X_features_poly[1]),2))-0.0012
poly_voting_mse = mean_absolute_percentage_error(np.exp(np.power(y_labels[1],2))-0.0012, visc_predictions)
poly_voting_rmse = np.sqrt(poly_voting_mse)
poly_voting_mse

0.03546833954618506

In [6]:
joblib.dump(poly_voting_reg, "poly_voting_reg.pkl")

['poly_voting_reg.pkl']

In [7]:

visc_predictions = np.exp(np.power(voting_reg.predict(X_features[1]),2))-0.0012
voting_mse = mean_absolute_percentage_error(np.exp(np.power(y_labels[1],2))-0.0012, visc_predictions)
voting_rmse = np.sqrt(voting_mse)
voting_mse

0.03483840084039444

In [8]:
joblib.dump(voting_reg, "voting_reg.pkl")

['voting_reg.pkl']

### Εδώ πραγματοποιούμε τη μέθοδο bagging για ένα δέντρο η οποία υπό συγκεκριμένες συνθήκες ταυτίζεται με τον random forest regressor.

In [9]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor(
    DecisionTreeRegressor( ), n_estimators=200,
    max_samples=5000, bootstrap=True)
bag_reg.fit(X_features_poly[0],y_labels[0])


BaggingRegressor(base_estimator=DecisionTreeRegressor(), max_samples=5000,
                 n_estimators=200)

In [10]:
joblib.dump(bag_reg, "bag_reg.pkl")

['bag_reg.pkl']

In [11]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(
    DecisionTreeRegressor(), n_estimators=200,
    learning_rate=0.5)
ada_reg.fit(X_features_poly[0],y_labels[0])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), learning_rate=0.5,
                  n_estimators=200)

In [12]:
joblib.dump(ada_reg, "ada_reg.pkl")

['ada_reg.pkl']

### Εδώ θα χρησιμοποιήσουμε τη μέθοδο του Boosting.

In [13]:
import xgboost
xgb_reg = xgboost.XGBRegressor(max_depth = 16,learning_rate=0.1)
xgb_reg.fit(X_features_poly[0],y_labels[0],
eval_set=[(X_features_poly[1],y_labels[1])])


[0]	validation_0-rmse:0.21653
[1]	validation_0-rmse:0.19721
[2]	validation_0-rmse:0.17996
[3]	validation_0-rmse:0.16471
[4]	validation_0-rmse:0.15126
[5]	validation_0-rmse:0.13944
[6]	validation_0-rmse:0.12920
[7]	validation_0-rmse:0.12028
[8]	validation_0-rmse:0.11251
[9]	validation_0-rmse:0.10589
[10]	validation_0-rmse:0.10016
[11]	validation_0-rmse:0.09527
[12]	validation_0-rmse:0.09120
[13]	validation_0-rmse:0.08780
[14]	validation_0-rmse:0.08486
[15]	validation_0-rmse:0.08249
[16]	validation_0-rmse:0.08049
[17]	validation_0-rmse:0.07888
[18]	validation_0-rmse:0.07759
[19]	validation_0-rmse:0.07646
[20]	validation_0-rmse:0.07553
[21]	validation_0-rmse:0.07482
[22]	validation_0-rmse:0.07423
[23]	validation_0-rmse:0.07383
[24]	validation_0-rmse:0.07355
[25]	validation_0-rmse:0.07325
[26]	validation_0-rmse:0.07302
[27]	validation_0-rmse:0.07283
[28]	validation_0-rmse:0.07270
[29]	validation_0-rmse:0.07263
[30]	validation_0-rmse:0.07256
[31]	validation_0-rmse:0.07255
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=16, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [14]:
joblib.dump(xgb_reg, "xgb_reg.pkl")

['xgb_reg.pkl']

In [15]:
import lightgbm as lgb
t_data = lgb.Dataset(X_features[0],label=y_labels[0])
v_data = lgb.Dataset(X_features[1],label=y_labels[1])

params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learning_rage': 0.5,
    'metric': {'l2','l1'},
    'verbose': -1
}
light = lgb.train(params,
                 train_set=t_data,
                 valid_sets=v_data,)
  

[LightGBM] [Warning] Unknown parameter: learning_rage
[1]	valid_0's l1: 0.1208	valid_0's l2: 0.0233265
[2]	valid_0's l1: 0.111866	valid_0's l2: 0.0201562
[3]	valid_0's l1: 0.10383	valid_0's l2: 0.0175885
[4]	valid_0's l1: 0.0967192	valid_0's l2: 0.0154761
[5]	valid_0's l1: 0.0903918	valid_0's l2: 0.0137443
[6]	valid_0's l1: 0.0848984	valid_0's l2: 0.0123378
[7]	valid_0's l1: 0.0799943	valid_0's l2: 0.0111518
[8]	valid_0's l1: 0.0757732	valid_0's l2: 0.010214
[9]	valid_0's l1: 0.0720889	valid_0's l2: 0.0094251
[10]	valid_0's l1: 0.0688827	valid_0's l2: 0.00877938
[11]	valid_0's l1: 0.0661609	valid_0's l2: 0.0082624
[12]	valid_0's l1: 0.0638519	valid_0's l2: 0.00783646
[13]	valid_0's l1: 0.0618409	valid_0's l2: 0.00747694
[14]	valid_0's l1: 0.0600776	valid_0's l2: 0.00717655
[15]	valid_0's l1: 0.0585724	valid_0's l2: 0.00693455
[16]	valid_0's l1: 0.0573665	valid_0's l2: 0.00674479
[17]	valid_0's l1: 0.0563107	valid_0's l2: 0.00657919
[18]	valid_0's l1: 0.0553822	valid_0's l2: 0.00644067


In [16]:
joblib.dump(light, "light.pkl")

['light.pkl']

In [17]:
from sklearn.ensemble import StackingRegressor
estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)]
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=RandomForestRegressor(n_estimators=100))

In [18]:
reg.fit(X_features[0], y_labels[0])

StackingRegressor(estimators=[('fr',
                               RandomForestRegressor(max_samples=5000,
                                                     n_estimators=150)),
                              ('svm', SVR(C=1, epsilon=0.001))],
                  final_estimator=RandomForestRegressor())

In [19]:
joblib.dump(reg, "stack.pkl")

['stack.pkl']